Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
def forward(x,w1,b1,w2,b2):
  h1 = tf.nn.relu(tf.matmul(x, w1) + b1)
  return tf.matmul(h1, w2) + b2


batch_size = 128

graph = tf.Graph()
with graph.as_default():
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_reg = tf.placeholder(tf.float32)  
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  w1 = tf.Variable(tf.truncated_normal([image_size * image_size, 1024]))
  b1 = tf.Variable(tf.zeros([1024]))
 
  w2 = tf.Variable(tf.truncated_normal([1024, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(forward(tf_train_dataset,w1,b1,w2,b2), tf_train_labels))
  
   
  loss += tf_reg * (tf.nn.l2_loss(w1)+tf.nn.l2_loss(w2))  
    
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(forward(tf_train_dataset,w1,b1,w2,b2))
  valid_prediction = tf.nn.softmax(forward(tf_valid_dataset,w1,b1,w2,b2))
  test_prediction = tf.nn.softmax(forward(tf_test_dataset,w1,b1,w2,b2))

In [6]:
num_steps = 3001
reg  = 0.01
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    idx = np.random.choice(train_labels.shape[0],batch_size)

    # Generate a minibatch.
    batch_data = train_dataset[idx, :]
    batch_labels = train_labels[idx, :]

    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,tf_reg : reg}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 3506.758301
Minibatch accuracy: 15.6%
Validation accuracy: 21.6%
Minibatch loss at step 500: 21.302298
Minibatch accuracy: 84.4%
Validation accuracy: 84.9%
Minibatch loss at step 1000: 0.784586
Minibatch accuracy: 87.5%
Validation accuracy: 83.7%
Minibatch loss at step 1500: 0.571575
Minibatch accuracy: 90.6%
Validation accuracy: 83.6%
Minibatch loss at step 2000: 0.882732
Minibatch accuracy: 80.5%
Validation accuracy: 83.9%
Minibatch loss at step 2500: 0.747778
Minibatch accuracy: 84.4%
Validation accuracy: 83.9%
Minibatch loss at step 3000: 0.654027
Minibatch accuracy: 85.9%
Validation accuracy: 83.6%
Test accuracy: 89.5%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [7]:
num_steps = 3001 #3001 #too slow

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  reduced_idx =  np.random.choice(train_labels.shape[0],batch_size*2)
  reduced_train_labels =  train_labels[reduced_idx] 
  reduced_train_dataset = train_dataset[reduced_idx]

  for step in range(num_steps):
    idx = np.random.choice(reduced_train_labels.shape[0],batch_size)

    # Generate a minibatch.
    batch_data = reduced_train_dataset[idx, :]
    batch_labels = reduced_train_labels[idx, :]

    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, tf_reg:0}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 390.161835
Minibatch accuracy: 9.4%
Validation accuracy: 27.1%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 71.5%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 71.5%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 71.5%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 71.5%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 71.5%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 71.5%
Test accuracy: 77.8%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [8]:
def forward(x,w1,b1,w2,b2,is_training=False):
  h1 = tf.nn.relu(tf.matmul(x, w1) + b1)
  keep_prop =  0.5 if is_training else 1
  h2 = tf.nn.dropout(h1,keep_prop)
  return tf.matmul(h2, w2) + b2


batch_size = 128

graph = tf.Graph()
with graph.as_default():
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_reg = tf.placeholder(tf.float32)  
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  w1 = tf.Variable(tf.truncated_normal([image_size * image_size, 1024]))
  b1 = tf.Variable(tf.zeros([1024]))
 
  w2 = tf.Variable(tf.truncated_normal([1024, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(forward(tf_train_dataset,w1,b1,w2,b2,True), tf_train_labels))
  
   
  loss += tf_reg * (tf.nn.l2_loss(w1)+tf.nn.l2_loss(w2))  
    
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(forward(tf_train_dataset,w1,b1,w2,b2))
  valid_prediction = tf.nn.softmax(forward(tf_valid_dataset,w1,b1,w2,b2))
  test_prediction = tf.nn.softmax(forward(tf_test_dataset,w1,b1,w2,b2))

In [9]:
num_steps = 3001 #3001 #too slow

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  reduced_idx =  np.random.choice(train_labels.shape[0],batch_size*2)
  reduced_train_labels =  train_labels[reduced_idx] 
  reduced_train_dataset = train_dataset[reduced_idx]

  for step in range(num_steps):
    idx = np.random.choice(reduced_train_labels.shape[0],batch_size)

    # Generate a minibatch.
    batch_data = reduced_train_dataset[idx, :]
    batch_labels = reduced_train_labels[idx, :]

    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, tf_reg:0}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 454.978943
Minibatch accuracy: 6.2%
Validation accuracy: 24.9%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.0%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.6%
Minibatch loss at step 1500: 0.234352
Minibatch accuracy: 100.0%
Validation accuracy: 73.6%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.8%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.7%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.4%
Test accuracy: 80.1%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [38]:
def forward(x,w1,b1,w2,b2,w3,b3,is_training=False):
  h1 = tf.nn.relu(tf.matmul(x, w1) + b1)
  if is_training:
     h1 = tf.nn.dropout(h1,0.7)
  h2 =  tf.nn.relu(tf.matmul(h1, w2) + b2)
  if is_training:
     h2 = tf.nn.dropout(h2,0.9)
  return tf.matmul(h2, w3) + b3


batch_size = 128

graph = tf.Graph()
with graph.as_default():
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_reg = tf.placeholder(tf.float32)  
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  w1 = tf.Variable(tf.truncated_normal([image_size * image_size, 1024], stddev=np.sqrt(2.0/ (image_size * image_size))))
  b1 = tf.Variable(tf.zeros([1024]))
 
  w2 = tf.Variable(tf.truncated_normal([1024, 500], stddev=np.sqrt(2.0/1024)))
  b2 = tf.Variable(tf.zeros([500]))

  w3 = tf.Variable(tf.truncated_normal([500, num_labels],stddev=np.sqrt(2.0 / 500)))
  b3 = tf.Variable(tf.zeros([num_labels]))
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(forward(tf_train_dataset,w1,b1,w2,b2,w3,b3,True), tf_train_labels))
  
   
  loss += reg * (tf.nn.l2_loss(w1)+ tf.nn.l2_loss(w2)+  tf.nn.l2_loss(w3))  
    
  # Optimizer.
#   global_step = tf.Variable(0)  # count the number of steps taken.
#   learning_rate = tf.train.exponential_decay(0.5, global_step, 1000, 0.96, staircase=True)
#   optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)   
  
  #optimizer = tf.train.GradientDescentOptimizer(0.03).minimize(loss)
  optimizer = tf.train.AdamOptimizer(1e-3).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(forward(tf_train_dataset,w1,b1,w2,b2,w3,b3))
  valid_prediction = tf.nn.softmax(forward(tf_valid_dataset,w1,b1,w2,b2,w3,b3))
  test_prediction = tf.nn.softmax(forward(tf_test_dataset,w1,b1,w2,b2,w3,b3))
    

In [39]:
num_steps = 5001
reg  = 1e-6
with tf.Session(graph=graph) as session:
  merged_summary_op = tf.merge_all_summaries()
  summary_writer = tf.train.SummaryWriter('/tmp/mnist_logs', session.graph)

  tf.initialize_all_variables().run()
  print("Initialized")
    
  for step in range(num_steps):
    idx = np.random.choice(train_labels.shape[0],batch_size)

    # Generate a minibatch.
    batch_data = train_dataset[idx, :]
    batch_labels = train_labels[idx, :]

    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,tf_reg : reg}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    

    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))

  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.543697
Minibatch accuracy: 10.9%
Validation accuracy: 31.5%
Minibatch loss at step 500: 0.513404
Minibatch accuracy: 84.4%
Validation accuracy: 85.3%
Minibatch loss at step 1000: 0.363018
Minibatch accuracy: 87.5%
Validation accuracy: 87.5%
Minibatch loss at step 1500: 0.234526
Minibatch accuracy: 92.2%
Validation accuracy: 87.8%
Minibatch loss at step 2000: 0.379569
Minibatch accuracy: 87.5%
Validation accuracy: 88.9%
Minibatch loss at step 2500: 0.329262
Minibatch accuracy: 89.8%
Validation accuracy: 89.2%
Minibatch loss at step 3000: 0.460997
Minibatch accuracy: 85.9%
Validation accuracy: 89.5%
Minibatch loss at step 3500: 0.291023
Minibatch accuracy: 90.6%
Validation accuracy: 89.2%
Minibatch loss at step 4000: 0.214385
Minibatch accuracy: 93.8%
Validation accuracy: 89.7%
Minibatch loss at step 4500: 0.291766
Minibatch accuracy: 90.6%
Validation accuracy: 89.6%
Minibatch loss at step 5000: 0.275406
Minibatch accuracy: 93.0%
Validation accurac